In [ ]:
!pip install recogym

# Likelihood based models

This notebook will outline the likelihood based approach to training on Bandit feedback.

Although before proceeding we will study the output of the simulator in a little more detail.

In [ ]:
from numpy.random.mtrand import RandomState
from recogym import Configuration
from recogym.agents import Agent
from sklearn.linear_model import LogisticRegression
from recogym import verify_agents
from recogym.agents import OrganicUserEventCounterAgent, organic_user_count_args
from recogym.evaluate_agent import verify_agents, plot_verify_agents

import gym, recogym
from copy import deepcopy
from recogym import env_1_args
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = [6, 3]

num_users = 5000
num_products = 10
num_samples = 20

env_1_args['phi_var'] = 0.0
env_1_args['number_of_flips'] = 0
env_1_args['sigma_mu_organic'] = 0.0
env_1_args['sigma_omega'] = 1
env_1_args['random_seed'] = 42
env_1_args['num_products'] = num_products
env_1_args['K'] = 5
env_1_args['number_of_flips'] = 5

env = gym.make('reco-gym-v1')
env.init_gym(env_1_args)

In [ ]:
data = deepcopy(env).generate_logs(num_users)

# Logistic Regression Model

## Turn Data into Features

Now we are going to build a _Logistic Regression_ model.

The model will predict _the probability of the click_ for the following data:
* _`Views`_ is a total amount of views of a particular _`Product`_ shown during _Organic_ _`Events`_ **before** a _Bandit_ _`Event`_.
* _`Action`_ is a proposed _`Product`_ at a _Bandit_ _`Event`_.

For example, assume that we have _`10`_ products. In _Organic_ _`Events`_, these products  were shown to a user as follows:
<table>
    <tr>
        <th>Product ID</th>
        <th>Views</th>
    </tr>
    <tr>
        <td>0</td>
        <td>0</td>
    </tr>
    <tr>
        <td>1</td>
        <td>0</td>
    </tr>
    <tr>
        <td>2</td>
        <td>0</td>
    </tr>
    <tr>
        <td>3</td>
        <td>7</td>
    </tr>
    <tr>
        <td>4</td>
        <td>0</td>
    </tr>
    <tr>
        <td>5</td>
        <td>0</td>
    </tr>
    <tr>
        <td>6</td>
        <td>0</td>
    </tr>
    <tr>
        <td>7</td>
        <td>8</td>
    </tr>
    <tr>
        <td>8</td>
        <td>11</td>
    </tr>
    <tr>
        <td>9</td>
        <td>0</td>
    </tr>
</table>

When we want to know the probability of the click for _`Product`_ = _`8`_ with available amounts of _`Views`_, the input data for the model will be:

_`0 0 0 7 0 0 0 0 8 11 0`_ _**`8`**_

The first 10 numbers are _`Views`_ of _`Products`_ (see above), the latest one is the _`Action`_.

The output will be two numbers:
* $0^{th}$ index: $1 - \mathbb{P}_c(P=p|V)$.
* $1^{st}$ index: $\mathbb{P}_c(P=p|V)$.

Here, $\mathbb{P}_c(P=p|V)$ is the probability of the click for a _`Product`_ $p$, provided that we have _`Views`_ $V$.


We will of course encode _`Action`_ using one hot encoding.
In our current example, the _`Action`_ is _`8`_. Thus, it is encoded as:

_`0 0 0 0 0 0 0 0`_ _**`1`**_ _`0`_

Here,
* Vector of _`Actions`_ has a size that is equal to the _*number of `Products`*_ i.e. _`10`_.
* _`Action`_ _`8`_ is marked as _`1`_ (_`Action`_ starts with _`0`_).

To build the final feature vector, we need to cross features from `Views` with `Action`.  

In [ ]:
from recogym.agents import FeatureProvider

class CountFeatureProvider(FeatureProvider):
    """Feature provider as an abstract class that defined interface of setting/getting features"""

    def __init__(self, config):
        super(CountFeatureProvider, self).__init__(config)
        self.feature_data = np.zeros((self.config.num_products))

    def observe(self, observation):
        """Consider an Organic Event for a particular user"""
        for session in observation.sessions():
            self.feature_data[int(session['v'])] += 1

    def features(self, observation):
        """Provide feature values adjusted to a particular feature set"""
        return self.feature_data

    def reset(self):
        self.feature_data = np.zeros((self.config.num_products))

In [ ]:
import math
import numpy as np
from recogym import Configuration, DefaultContext, Observation
from recogym.envs.session import OrganicSessions

def build_train_data(logs, feature_provider):
    user_states, actions, rewards, proba_actions = [], [], [], []
    
    current_user = None
    for _, row in logs.iterrows():
        if current_user != row['u']:
            # Use has changed: start a new session and reset user state
            current_user = row['u']
            sessions = OrganicSessions()
            feature_provider.reset()
        
        context = DefaultContext(row['u'], row['t'])
        
        if row['z'] == 'organic':
            sessions.next(context, row['v'])
            
        else:
            # For each bandit event, generate one observation for the user state, the taken action
            # the obtained reward and the used probabilities
            feature_provider.observe(Observation(context, sessions))
            user_states.append(feature_provider.features(None))
            actions.append(row['a'])
            rewards.append(row['c'])
            proba_actions.append(row['ps'])
            
            # Start a new organic session
            sessions = OrganicSessions()
    
    return np.array(user_states), np.array(actions).astype(int), np.array(rewards), np.array(proba_actions)

In [ ]:
# You can now see data that will be provided to our agents based on logistic regressions
config = Configuration(env_1_args)
count_feature_provider = CountFeatureProvider(config=config)

user_states, actions, rewards, proba_actions = build_train_data(data, count_feature_provider)

In [ ]:
preview_start, preview_size = 500, 3

print('user product views count at action time')
print(user_states[preview_start:preview_start + preview_size])
print('taken actions', actions[preview_start:preview_start + preview_size])
print('obtained rewards', rewards[preview_start:preview_start + preview_size])
print('probablities of the taken actions', proba_actions[preview_start:preview_start + preview_size])

Look at the data and see how it maps into the features - which is the combination of the history and the actions and the label which is clicks.  Note that only the bandit events correspond to records in the training data.

In order to do personalisation it is necessary to cross the action and history features. _Why_?  We do the simplest possible cross an element wise kronecker product.


In [ ]:
class LikelihoodAgent(Agent):
    def __init__(self, feature_provider, use_argmax=False, seed=43):
        self.feature_provider = feature_provider
        self.use_argmax = use_argmax
        self.random_state = RandomState(seed)
        self.model = None
        
    @property
    def num_products(self):
        return self.feature_provider.config.num_products
    
    def _create_features(self, user_state, action):
        """Create the features that are used to estimate the expected reward from the user state.
        """
        features = np.zeros(len(user_state) * self.num_products)
        features[action * len(user_state): (action + 1) * len(user_state)] = user_state
        
        return features
    
    def train(self, logs):
        user_states, actions, rewards, proba_actions = build_train_data(logs, self.feature_provider)
        
        features = np.vstack([
            self._create_features(user_state, action) 
            for user_state, action in zip(user_states, actions)
        ])
        self.model = LogisticRegression(solver='lbfgs', max_iter=5000)
        self.model.fit(features, rewards)
    
    def _score_products(self, user_state):
        all_action_features = np.array([
            self._create_features(user_state, action) 
            for action in range(self.num_products)
        ])
        return self.model.predict_proba(all_action_features)[:, 1]
        
    def act(self, observation, reward, done):
        """Act method returns an action based on current observation and past history"""
        self.feature_provider.observe(observation)        
        user_state = self.feature_provider.features(observation)
        prob = self._score_products(user_state)
        
        if not self.use_argmax:
            action = self.random_state.choice(self.num_products, p=(prob / np.sum(prob)))
            ps = prob[action]
            all_ps = prob.copy()
        else:
            action = np.argmax(prob)
            ps = 1.0
            all_ps = np.zeros(self.num_products)
            all_ps[action] = 1.0
      
        return {
            **super().act(observation, reward, done),
            **{
                'a': action,
                'expected-value': prob[action],
                'ps': ps,
                'ps-a': all_ps,
            }
        }

    def reset(self):
        self.feature_provider.reset()  

In [ ]:
# Have a look at the feature vector used by the Likelihood agent
picked_sample = 500

count_product_views_feature_provider = CountFeatureProvider(config)
likelihood_logreg = LikelihoodAgent(count_product_views_feature_provider)

print('User state', user_states[picked_sample])
print('action', actions[picked_sample])
print('Created cross features')
print(likelihood_logreg._create_features(user_states[picked_sample], actions[picked_sample]))

In [ ]:
%%time

likelihood_logreg = LikelihoodAgent(count_product_views_feature_provider, use_argmax=True)
likelihood_logreg.train(data)

In [ ]:
organic_counter_agent = OrganicUserEventCounterAgent(Configuration({
            **organic_user_count_args,
            **env_1_args,
            'select_randomly': True,
        }))

In [ ]:
result = verify_agents(env, 
                       number_of_users = 5000, 
                       agents = {
                           'likelihood logreg': likelihood_logreg, 
                           'organic count': organic_counter_agent
                       })

In [ ]:
fig = plot_verify_agents(result)
plt.show()

# Embeddings and retrieval

Now to illustrate the use of embeddings and then a nearest neighbour retrieval approach, we are going to use organic data, build a matrix of product coviews by the same user, and factorize this matrix with an SVD to get product embeddings.

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import LabelBinarizer

def create_embeddings(data, num_products, embedding_size=5):
    organic_events = data[data['z'] == 'organic']
    print("num organic events", len(organic_events))

    n_users = data['u'].nunique()
    counts = np.zeros((n_users, num_products))
    print("num users", n_users)

    binarizer = LabelBinarizer().fit(np.arange(num_products))
    for u in range(n_users):
        binarized_views_of_user = binarizer.transform(organic_events[organic_events['u'] == u]['v'])
        counts[u, :] = binarized_views_of_user.sum(axis=0)

    counts_above_zero = 1. * (counts > 0)  # above zero counts only
    co_counts = np.matmul(counts_above_zero.T, counts_above_zero)
    print("Coview matrix shape", co_counts.shape)

    embeddings = TruncatedSVD(embedding_size).fit_transform(co_counts)
    print("Embeddings shape", embeddings.shape)
    return embeddings

In [ ]:
embeddings = create_embeddings(data, num_products)

In [ ]:
from scipy import spatial

class RetrievalAgent(Agent):
    def __init__(self, env, feature_provider, product_embeddings):
        # Set environment as an attribute of Agent.
        self.env = env
        self.feature_provider = feature_provider
        self.product_embeddings = product_embeddings
        self.num_products = self.product_embeddings.shape[0]
        self.embeddings_index = spatial.KDTree(self.product_embeddings)
        self.reset()
        
    def build_user_embedding(self, user_state):
        return np.average(self.product_embeddings, axis=0, weights=user_state)

    def act(self, observation, reward, done):
        """Act method returns an action based on current observation and past history"""
        self.feature_provider.observe(observation)
        user_embedding = self.build_user_embedding(self.feature_provider.features(observation))
        closest_product = self.embeddings_index.query(user_embedding)[1]
        scores = np.zeros(self.num_products)
        scores[closest_product] = 1.0
        return {
            **super().act(observation, reward, done),
            **{
                'a': closest_product,
                'ps': 1.,
                'ps-a': scores,
            }
        }

    def reset(self):
        self.feature_provider.reset()

In [ ]:
def build_retrieval_agent(env_args, embeddings):
    config = Configuration(env_args)
    retrieval_agent = RetrievalAgent(
        config,
        CountFeatureProvider(config),
        embeddings)
    
    return retrieval_agent

retrieval_agent = build_retrieval_agent(env_1_args, embeddings)

In [ ]:
result = verify_agents(env, 
                       number_of_users = 5000, 
                       agents = {
                           'likelihood logreg': likelihood_logreg, 
                           'organic count': organic_counter_agent,
                           'retrieval_agent': retrieval_agent
                       })

In [ ]:
fig = plot_verify_agents(result)
plt.show()